# DHIS Medical Image Processing - Django Integration Demo

This notebook demonstrates how to interact with the Django backend for the DHIS medical image processing system.

## 1. Setup Django Environment

In [ ]:
import os
import sys
import django

# Add the backend directory to the Python path
sys.path.insert(0, '/app')

# Configure Django settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'image_processor.settings')
django.setup()

print("Django version:", django.get_version())
print("Django setup complete!")

## 2. Import Django Models and Utilities

In [ ]:
from api.models import ImageUpload
from api.utils import LLMService
from django.conf import settings
import pandas as pd
import json
from datetime import datetime, timedelta

print("Models and utilities imported successfully!")
print(f"\nDatabase: {settings.DATABASES['default']['ENGINE']}")
print(f"Media Root: {settings.MEDIA_ROOT}")
print(f"Portkey Model: {settings.PORTKEY_MODEL}")

## 3. Query Existing Image Uploads

In [ ]:
# Get all image uploads
uploads = ImageUpload.objects.all()
print(f"Total uploads: {uploads.count()}")

# Display recent uploads
if uploads.exists():
    recent_uploads = uploads.order_by('-uploaded_at')[:5]
    
    data = []
    for upload in recent_uploads:
        data.append({
            'ID': upload.id,
            'Filename': upload.original_filename,
            'Status': upload.processing_status,
            'Uploaded': upload.uploaded_at.strftime('%Y-%m-%d %H:%M'),
            'Has Results': bool(upload.extracted_data)
        })
    
    df = pd.DataFrame(data)
    print("\nRecent Uploads:")
    display(df)
else:
    print("No uploads found yet.")

## 4. Analyze Extracted Data

In [ ]:
# Analyze extracted medical data
processed_uploads = ImageUpload.objects.filter(processing_status='completed')

if processed_uploads.exists():
    extracted_data_list = []
    
    for upload in processed_uploads:
        if upload.extracted_data:
            try:
                data = json.loads(upload.extracted_data) if isinstance(upload.extracted_data, str) else upload.extracted_data
                data['upload_id'] = upload.id
                data['filename'] = upload.original_filename
                extracted_data_list.append(data)
            except json.JSONDecodeError:
                continue
    
    if extracted_data_list:
        df_extracted = pd.DataFrame(extracted_data_list)
        print("Extracted Medical Information:")
        display(df_extracted[['upload_id', 'filename', 'first_name', 'last_name', 'date_of_birth', 'date_of_diagnosis']].head())
        
        # Case detection options analysis
        if 'case_detection_options' in df_extracted.columns:
            all_options = []
            for options in df_extracted['case_detection_options'].dropna():
                if isinstance(options, list):
                    all_options.extend(options)
            
            if all_options:
                print("\nCase Detection Options Summary:")
                option_counts = pd.Series(all_options).value_counts()
                display(option_counts)
    else:
        print("No extracted data found.")
else:
    print("No processed uploads found.")

## 5. Test LLM Service

In [ ]:
# Initialize LLM service
llm_service = LLMService()

# Test with a sample medical text
sample_text = """
Patient: John Doe
Date of Birth: 15/03/1985
Diagnosis Date: 10/01/2024
Condition: Hypertension
"""

print("Testing LLM extraction with sample text...")
print("Sample text:")
print(sample_text)

try:
    # Extract information using LLM
    result = llm_service.extract_medical_info_from_text(sample_text)
    print("\nExtracted Information:")
    print(json.dumps(result, indent=2))
except Exception as e:
    print(f"Error testing LLM: {e}")
    print("Make sure Portkey API keys are configured in .env file")

## 6. Database Statistics

In [ ]:
from django.db.models import Count, Q
from django.utils import timezone

# Get upload statistics
stats = ImageUpload.objects.aggregate(
    total=Count('id'),
    pending=Count('id', filter=Q(processing_status='pending')),
    processing=Count('id', filter=Q(processing_status='processing')),
    completed=Count('id', filter=Q(processing_status='completed')),
    failed=Count('id', filter=Q(processing_status='failed'))
)

print("Upload Statistics:")
for key, value in stats.items():
    print(f"  {key.capitalize()}: {value}")

# Recent activity (last 7 days)
seven_days_ago = timezone.now() - timedelta(days=7)
recent_count = ImageUpload.objects.filter(uploaded_at__gte=seven_days_ago).count()
print(f"\nUploads in last 7 days: {recent_count}")

## 7. Create a Test Upload (Optional)

In [ ]:
# Uncomment to create a test upload
# from django.core.files.base import ContentFile
# import base64

# # Create a test image upload
# test_upload = ImageUpload(
#     original_filename="test_medical_report.jpg",
#     processing_status="pending"
# )
# 
# # Create a dummy file (1x1 white pixel PNG)
# dummy_image = base64.b64decode("iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAAAADUlEQVR42mP8/5+hHgAHggJ/PchI7wAAAABJRU5ErkJggg==")
# test_upload.image.save("test.png", ContentFile(dummy_image), save=True)
# 
# print(f"Created test upload with ID: {test_upload.id}")

## 8. Export Data to CSV

In [ ]:
# Export all uploads to CSV
if ImageUpload.objects.exists():
    exports = []
    for upload in ImageUpload.objects.all():
        row = {
            'id': upload.id,
            'filename': upload.original_filename,
            'status': upload.processing_status,
            'uploaded_at': upload.uploaded_at.isoformat(),
            'processed_at': upload.processed_at.isoformat() if upload.processed_at else None,
        }
        
        # Add extracted data if available
        if upload.extracted_data:
            try:
                data = json.loads(upload.extracted_data) if isinstance(upload.extracted_data, str) else upload.extracted_data
                row.update(data)
            except:
                pass
        
        exports.append(row)
    
    df_export = pd.DataFrame(exports)
    
    # Save to CSV
    csv_path = '/app/notebooks/dhis_exports.csv'
    df_export.to_csv(csv_path, index=False)
    print(f"Data exported to: {csv_path}")
    print(f"Total records: {len(df_export)}")
else:
    print("No data to export.")